In [ ]:
from mvnc import mvncapi as mvnc

In [ ]:
devices = mvnc.enumerate_devices()
devices

In [ ]:
device = mvnc.Device(devices[0])
device.open()

In [ ]:
import os
h = 52
w = 74
model_dir = os.path.join(os.environ.get('SRC_DIR',''),'movidius')
pnet_file = os.path.join(model_dir,'pnet-{}x{}.graph'.format(h,w))
onet_file = os.path.join(model_dir,'onet.graph')
rnet_file = os.path.join(model_dir,'rnet.graph')

In [ ]:
with open(pnet_file, mode='rb') as f:
    graphData = f.read()
pnetGraph = mvnc.Graph('pnet')
pnetIn, pnetOut = pnetGraph.allocate_with_fifos(device, graphData)

In [ ]:
with open(rnet_file, mode='rb') as f:
    graphData = f.read()
rnetGraph = mvnc.Graph('rnet')
rnetIn, rnetOut = rnetGraph.allocate_with_fifos(device, graphData)

In [ ]:
with open(onet_file, mode='rb') as f:
    graphData = f.read()
onetGraph = mvnc.Graph('onet')
onetIn, onetOut = onetGraph.allocate_with_fifos(device, graphData)

In [ ]:
def _rnet_proxy(img):
    rnetGraph.queue_inference_with_fifo_elem(rnetIn, rnetOut, img, 'rnet')
    output, userobj = rnetOut.read_elem()
    return output

def _onet_proxy(img):
    onetGraph.queue_inference_with_fifo_elem(onetIn, onetOut, img, 'onet')
    output, userobj = onetOut.read_elem()
    return output

In [ ]:
def _pnet_proxy(img):
    pnetGraph.queue_inference_with_fifo_elem(pnetIn, pnetOut, img, 'pnet')
    output, userobj = pnetOut.read_elem()
    return output

In [ ]:
pnets=[(_pnet_proxy,h,w)]

In [ ]:
import cv2
import numpy as np
frame = cv2.imread(os.path.join(model_dir,'test.png')).astype(np.float32)

In [ ]:
if (frame.shape[1] != 640) or (frame.shape[0] != 480):
    frame = cv2.resize(frame, (640, 480), interpolation=cv2.INTER_AREA)
%matplotlib inline
from matplotlib.pyplot import imshow
imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.uint8))

In [ ]:
rgb_frame = frame[:, :, ::-1]

In [ ]:
import tensorflow as tf
sess = tf.Session()

In [ ]:
import sys
sys.path.append(os.environ.get('SRC_DIR',''))
import align.detect_face as detect_face
pnets_proxy, rnet, onet = detect_face.create_movidius_mtcnn(sess,os.path.join(os.environ.get('SRC_DIR',''),'align'),
                                                            pnets, _rnet_proxy, _onet_proxy)

In [ ]:
bounding_boxes, _ = detect_face.movidius_detect_face(rgb_frame, pnets_proxy, rnet, onet, [0.7, 0.7, 0.7])
bounding_boxes

In [ ]:
if bounding_boxes is not None:
    for face in bounding_boxes:
        face_bb = face.astype(int)
        cv2.rectangle(frame,(face_bb[0], face_bb[1]), (face_bb[2], face_bb[3]),(0, 255, 0), 2)
        
%matplotlib inline
from matplotlib.pyplot import imshow
imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.uint8))

In [ ]:
rnetIn.destroy()
rnetOut.destroy()
rnetGraph.destroy()
onetIn.destroy()
onetOut.destroy()
onetGraph.destroy()
pnetIn.destroy()
pnetOut.destroy()
pnetGraph.destroy()
device.close()